In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, AvgPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
SEED = np.random.seed(69)
PATH = '/content/drive/My Drive/Colab Notebooks/AiDesignerBot/'

In [4]:
def load_data():
    labels = pd.read_csv(PATH + 'interiors_clf_sample.csv')

    train_datagen = ImageDataGenerator(validation_split=.25, rescale=1/255., horizontal_flip=True, vertical_flip=True,
                                       rotation_range=45)
    train_data = train_datagen.flow_from_dataframe(dataframe=labels, directory=PATH + 'images/',
                                                   x_col='title', y_col='target', target_size=(225, 225),
                                                   batch_size=32, class_mode='raw', subset='training', seed=SEED)
    
    test_datagen = ImageDataGenerator(validation_split=.25, rescale=1/255.)
    test_data = test_datagen.flow_from_dataframe(dataframe=labels, directory=PATH + 'images/',
                                                 x_col='title', y_col='target', target_size=(225, 225),
                                                 batch_size=32, class_mode='raw', subset='validation', seed=SEED)
    
    return train_data, test_data

In [5]:
def create_model(input_shape):
    model = Sequential()
    optimizer = Adam(lr=1e-3)
    model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same', input_shape=input_shape, activation='relu'))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=12, kernel_size=(5, 5), padding='valid', activation='relu'))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='valid', activation='relu'))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    return model

In [6]:
def train_model(model, train_data, test_data, batch_size=None, epochs=10,
               steps_per_epoch=None, validation_steps=None):
    if steps_per_epoch == None:
        steps_per_epoch = len(train_data)
    if validation_steps == None:
        validation_steps = len(test_data)

    model.fit(train_data, validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              verbose=1, shuffle=True, validation_steps=validation_steps)

    return model

In [7]:
model = create_model((225, 225, 3),)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 225, 225, 6)       456       
_________________________________________________________________
average_pooling2d (AveragePo (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 12)      1812      
_________________________________________________________________
average_pooling2d_1 (Average (None, 54, 54, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        9632      
_________________________________________________________________
average_pooling2d_2 (Average (None, 25, 25, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0

In [8]:
train, test = load_data()

Found 533 validated image filenames.
Found 177 validated image filenames.


In [9]:
pretrained_model = train_model(model, train, test, epochs=10)

Epoch 1/10
17/17 [==============================] - 25s 1s/step - loss: 0.7295 - acc: 0.4934 - val_loss: 0.6820 - val_acc: 0.4972
Epoch 2/10
17/17 [==============================] - 21s 1s/step - loss: 0.6870 - acc: 0.5338 - val_loss: 0.6527 - val_acc: 0.5480
Epoch 3/10
17/17 [==============================] - 21s 1s/step - loss: 0.6223 - acc: 0.6602 - val_loss: 0.5224 - val_acc: 0.7571
Epoch 4/10
17/17 [==============================] - 21s 1s/step - loss: 0.5446 - acc: 0.7189 - val_loss: 0.4935 - val_acc: 0.7740
Epoch 5/10
17/17 [==============================] - 21s 1s/step - loss: 0.5232 - acc: 0.7432 - val_loss: 0.5882 - val_acc: 0.6949
Epoch 6/10
17/17 [==============================] - 21s 1s/step - loss: 0.4428 - acc: 0.8011 - val_loss: 0.4994 - val_acc: 0.7740
Epoch 7/10
17/17 [==============================] - 21s 1s/step - loss: 0.4393 - acc: 0.8050 - val_loss: 0.4682 - val_acc: 0.8023
Epoch 8/10
17/17 [==============================] - 21s 1s/step - loss: 0.3580 - acc: 0.86

In [10]:
pretrained_model.save('interiors_clf_model')

INFO:tensorflow:Assets written to: interiors_clf_model/assets


In [11]:
!cp -r interiors_clf_model '/content/drive/My Drive/Colab Notebooks/AiDesignerBot/'